In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential

from keras.layers import Dense, Dropout
# from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [21]:
df_train = pd.read_csv('./data/asilomar_nov182017_all.csv')
# this is actually [u1, v1, u2, v2]
X = df_train[['u','v','w','h']]
y = df_train['distance']

In [22]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.30)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [23]:
NUM_FEATURES = len(X.columns)

In [32]:
def dist_regression_model():
    model = Sequential()
    model.add(Dense(NUM_FEATURES*4, input_dim=NUM_FEATURES, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [33]:
model = dist_regression_model()

In [34]:
X_np = X_train
print(X_np.shape)
y_np = y_train
print(y_np.shape)

(11088, 4)
(11088,)


In [35]:
model.fit(x=X_np, y=y_np,batch_size=32, epochs=15)

Epoch 1/15
11088/11088 [==============================] - 1s 106us/step - loss: 932.3553
Epoch 2/15
11088/11088 [==============================] - 1s 88us/step - loss: 155.2218
Epoch 3/15
11088/11088 [==============================] - 1s 89us/step - loss: 143.3422
Epoch 4/15
11088/11088 [==============================] - 1s 89us/step - loss: 133.4575
Epoch 5/15
11088/11088 [==============================] - 1s 88us/step - loss: 127.1159
Epoch 6/15
11088/11088 [==============================] - 1s 89us/step - loss: 119.8170
Epoch 7/15
11088/11088 [==============================] - 1s 89us/step - loss: 116.4159
Epoch 8/15
11088/11088 [==============================] - 1s 90us/step - loss: 112.6494
Epoch 9/15
11088/11088 [==============================] - 1s 96us/step - loss: 111.3344
Epoch 10/15
11088/11088 [==============================] - 1s 91us/step - loss: 109.5190
Epoch 11/15
11088/11088 [==============================] - 1s 90us/step - loss: 109.0823
Epoch 12/15
11088/11088 [====

In [36]:
# training error
pred = model.predict(X_np)

error = y_np[:,np.newaxis]-pred
print np.mean(np.square(error))

# mean absolute percentage
mean_abs_percentage = np.mean(np.abs(error)/y_np[:,np.newaxis])
print mean_abs_percentage



101.576695473
0.122887497203


In [37]:
# test set
X_val_np = X_val
y_val_np = y_val
pred = model.predict(X_val_np)

# mean square error
error = y_val_np[:,np.newaxis]-pred
print np.mean(np.square(error))

# mean absolute percentage
mean_abs_percentage = np.mean(np.abs(error)/y_val_np[:,np.newaxis])
print mean_abs_percentage

97.1854886608
0.120824845092


In [38]:
print error[:10]

[[  0.69305062]
 [  1.40190018]
 [-47.57186915]
 [ -2.92650081]
 [  6.01505701]
 [  0.17678195]
 [ -2.32646895]
 [  5.97551605]
 [ -2.25056285]
 [ -0.76839642]]


In [51]:
bbox = np.array([[147.,  295.,147+331.,295.+173.]],np.float32)
bbox = scaler.transform(bbox)
print bbox.shape
print model.predict(bbox)

(1, 4)
[[ 17.30764198]]


In [47]:
scaler.inverse_transform(X_train[100,:])


array([ 256.,  243.,  358.,  337.])

In [42]:
bbox = np.array([[240,300,46,39]],np.float32)
scaler.transform(bbox)

array([[-0.14987808,  4.66555023, -4.74805546, -5.16080523]], dtype=float32)